# MIxS 5 Package Terms
This notebooks creates two spreadsheets:
- A spreadsheet of package terms, but excludes any package terms that are also the core terms.
- A spreadsheet of the package only terms (as above), but also excludes the package name and other non-essential columns.

In [1]:
import pandas as pds

## load MIxS 5 package and core terms
note the use of `engine='openpyxl'`. This is required due to a security flaw in the older `xlrd` package.  
see: https://stackoverflow.com/questions/65254535/xlrd-biffh-xlrderror-excel-xlsx-file-not-supported

In [104]:
df_core = pds.read_excel('data/mixs_v5.xlsx', sheet_name='MIxS', engine='openpyxl')
df_package = pds.read_excel('data/mixs_v5.xlsx', sheet_name='environmental_packages', engine='openpyxl')

In [105]:
# df_package.head() # uncomment to peek at data

In [106]:
# df_core # uncomment to peek at data

## perform some data clean up
- drop rows with all NaNs (created by blank rows in source spreadsheet)
- strip spaces from around string values

In [107]:
df_core.dropna(how='all', inplace=True)
df_package.dropna(how='all', inplace=True)

In [109]:
df_core = df_core.applymap(lambda x: x.strip() if type(x) == type('') else x)
df_package = df_package.applymap(lambda x: x.strip() if type(x) == type('') else x)

## check overlap between package and core terms
steps:
- create unique list of package and core terms
- count number of package terms that are not core terms
- count number of package terms that are core terms

**result**: In the MIxS 5 spreadsheet, 3 terms are package and core terms: 'alt', 'elev', 'depth'

In [110]:
core_terms = list(df_core['Structured comment name'].unique())
package_terms = list(df_package['Structured comment name'].unique())
print('number of unique core terms:', len(core_terms))
print('number of unique package terms:', len(package_terms))

number of unique core terms: 94
number of unique package terms: 509


In [111]:
package_only_terms = [term for term in package_terms if term not in core_terms]
print('number of package terms not in core terms:', len(package_only_terms))

number of package terms not in core terms: 506


In [112]:
package_core_terms = [term for term in package_terms if term in core_terms]
print('number of package terms are also core terms:', len(package_core_terms))
package_core_terms

number of package terms are also core terms: 3


['alt', 'elev', 'depth']

## create spreadsheet excluding the core terms (i.e., package only terms)

In [119]:
df_package_no_core = df_package[ df_package['Structured comment name'].isin(package_only_terms) ]
print('number of unfiltered rows:', len(df_package))
print('number of filtered rows:', len(df_package_no_core))

number of unfiltered rows: 1013
number of filtered rows: 993


In [116]:
# df_filtered

In [149]:
df_package_no_core.to_csv('output/mixs5-packages-exclude-core.tsv', sep='\t')

## create spreadsheet of package only terms, but drop the package name and other non-essential columns

In [147]:
columns_to_drop = ['Environmental package', 'MIXS ID', 'Position', 'Occurrence', 'Requirement', 'Example', 'Preferred unit', 'Expected value', 'Value syntax']
df_package_only_terms = df_package_no_core.drop(columns=columns_to_drop).drop_duplicates()

In [148]:
df_package_only_terms

,Structured comment name,Package item,Definition
2,barometric_press,barometric pressure,Force per unit area exerted against a surface ...
3,carb_dioxide,carbon dioxide,Carbon dioxide (gas) amount or concentration a...
4,carb_monoxide,carbon monoxide,Carbon monoxide (gas) amount or concentration ...
5,chem_administration,chemical administration,List of chemical compounds administered to the...
6,humidity,humidity,"Amount of water vapour in the air, at the time..."
...,...,...,...
999,soluble_react_phosp,soluble reactive phosphorus,Concentration of soluble reactive phosphorus
1002,suspend_part_matter,suspended particulate matter,Concentration of suspended particulate matter
1006,tot_diss_nitro,total dissolved nitrogen,"Total dissolved nitrogen concentration, report..."
1007,tot_inorg_nitro,total inorganic nitrogen,Total inorganic nitrogen content


In [150]:
df_package_only_terms.to_csv('output/mixs5-packages-terms-only.tsv', sep='\t')